In [65]:
import json 
import ijson
import pandas as pd
import re, os
import random
from collections import Counter

In [66]:
 
country_code_df= pd.read_csv('database/country_code.csv' ,sep = ',')
country_code = country_code_df.values.tolist()
#if "iran" in country_code_df.values:
#    index_row = [country_code.index(row) for row in country_code if "iran" in row]
#    print(country_code[index_row[0]][1])

domain_df= pd.read_csv('database/domain.csv' ,sep = '.')
domain_code = domain_df.values.tolist()
#print(domain_code)
#if "ae" in domain_df.values:
#    index_row = [domain_code.index(row) for row in domain_code if "ae" in row]
#    print(domain_code[index_row[0]][1])

    
flags_df= pd.read_csv('database/flags_new.csv' ,sep = ',')
flags = flags_df.values.tolist()
#print(flags)
#if '🇧🇭' in flags_df.values:
#    index_row = [flags.index(row) for row in flags if '🇧🇭' in row]
#    print(flags[index_row[0]][1])


language_code_df= pd.read_csv('database/language_code.csv' ,sep = ':')
language_code = language_code_df.values.tolist()
#print(language_code)
#if 'fa' in language_code_df.values:
#    index_row = [language_code.index(row) for row in language_code if "fa" in row]
#    print(index_row)
#    print(language_code[index_row[0]][0])
    
cities_df= pd.read_csv('database/worldcities.csv' ,sep = ',')
cities = cities_df.values.tolist()
#if 'addis' in cities_df.values:
#    index_row = [cities.index(row) for row in cities if "addis" in row]
#    print(cities[index_row[0]][5])

country_to_code = [row[1] for row in country_code]
country_class = { name: k+1 for k,name in enumerate(country_to_code) }
#print(country_class["af"])

In [67]:
def tokenize(s): return re.sub('([^A-Za-z0-9 ]+)', ' \\1 ', s).split()  # add space around anything not alphanum
def sanify(s): return s.replace(':',';').replace('|','/')
def writeToVWFile(filename, examples):
    with open(filename, 'w') as h:
        for ex in examples:
            h.write(ex.strip()+'\n')
def pop_2(tokens,n=2):# 
    pop_list =[]
    for ii in tokens:
        if len(ii)<=n:
            pop_list.append(ii)
    for ii in pop_list:
        tokens.remove(ii)
    return tokens
    
def check_flag(tokens):
    labeled = False
    label = None
    #tokens = pop_2(tokens)
    for token in tokens: #check for countries with one part
        if token in flags_df.values:
            index_row = [flags.index(row) for row in flags if token in row[0]]
            if index_row:
                label_char =flags[index_row[0]][1]
                label_char =str(label_char).lower()
                label = country_class[label_char]
                labeled =True
                break
    return labeled , label

def city_country_check(place):
    is_labeled = False
    label = None
    place = pop_2(place)
    for token in place: #check for countries with one part
        if token in country_code_df.values:
            index_row = [country_code.index(row) for row in country_code if token in row[0]]
            label_char =country_code[index_row[0]][1]
            label_char =str(label_char).lower()
            label = country_class[label_char]
            is_labeled= True
            break
    if not is_labeled:
        for k in range(len(place)-1): #for cities like new york or mexico city
            token ='%s %s' % (place[k],place[k+1])
            token = str(token).lower()
            #print(token)
            if token in cities_df.values:
                index_row = [cities.index(row) for row in cities if token in row]
                label_char =cities[index_row[0]][5]
                label_char =str(label_char).lower()
                label = country_class[label_char]
                is_labeled= True
                break
    if not is_labeled:
        for token in place: #check for cities with one part
            if token in cities_df.values:
                index_row = [cities.index(row) for row in cities if token in row]
                label_char=cities[index_row[0]][5]
                label_char =str(label_char).lower()
                label = country_class[label_char]
                is_labeled= True
                break
    return is_labeled , label

def operation(data):
    examples = []
    labels_list = []
    for i in range(len(data)):
        label = None
        namespaces = {}
        is_labeled= False
        if data[i]["place"] != None:
            #print(data[i]["place"])
            label_char = data[i]["place"]['country_code']
            label_char =str(label_char).lower()
            namespaces['p'] = [label_char]
            label = country_class[label_char]
            is_labeled= True

        if data[i]["user"]["location"] != "":
            place= tokenize(data[i]["user"]["location"].strip())
            place=pop_2(place)
            text = []
            text += place
            #print(place)
            namespaces['l'] = text
            if not is_labeled:
                is_labeled , label =city_country_check(place)

        if data[i]["user"]["name"] != "":
            user_name= tokenize(data[i]["user"]["name"].strip())
            namespaces['n'] = user_name
            if not is_labeled:
                is_labeled , label = check_flag(user_name)
            if not is_labeled:
                is_labeled , label =city_country_check(user_name)

        if data[i]["user"]["screen_name"] != "":
            screen_name= tokenize(data[i]["user"]["screen_name"].strip())
            namespaces['s'] = screen_name
            if not is_labeled:
                is_labeled , label = check_flag(screen_name)
            if not is_labeled:
                is_labeled , label =city_country_check(screen_name)

        if data[i]["user"]["description"] != "":
            description= tokenize(data[i]["user"]["description"].strip())
            namespaces['d'] = description
            if not is_labeled:
                is_labeled , label = check_flag(description)
            if not is_labeled:
                is_labeled , label =city_country_check(description)

        if data[i]["full_text"] != "":
            full_text= tokenize(data[i]["full_text"].strip())
            full_text = pop_2(full_text , 1)
            namespaces['f'] = full_text
            if not is_labeled:
                is_labeled , label = check_flag(full_text)

        if not is_labeled:
            if 'url' in data[i]["user"]["entities"]:
                if 'display_url' in data[i]["user"]["entities"]['url']['urls'][0]:
                    if data[i]["user"]["entities"]['url']['urls'][0]['display_url'] != 'None':
                        url = data[i]["user"]["entities"]['url']['urls'][0]['display_url']
                        #print(url)
                        x = re.search("(\.[^c][^o]($|\W))", url)
                        #print(x)
                        if x!=None:
                            x = re.search("(\w\w)", x[0])
                            if x!=None:
                                if x[0] in country_code_df.values:
                                    namespaces['m'] = x[0] 
        if data[i]["lang"] != "":
            lang= tokenize(data[i]["lang"].strip())
            #print(lang)
            if lang[0] in language_code_df.values:
                index_row = [language_code.index(row) for row in language_code if lang[0] in row]
                #print(language_code[index_row[0]][0])
                namespaces['g'] = [language_code[index_row[0]][0]]
        if "p" in namespaces.keys():
            b = namespaces["p"]
            #print(namespaces["p"])
        if label!=None:
            labels_list.append(label)
            ex = ''
            for ns,words in namespaces.items():
                ex += '%s %s' % (ns, ' '.join(map(sanify,words)))
            examples += [ex]
    return examples , labels_list

def write_files(file_name,examples):
    random.seed(9999)
    random.shuffle(examples)
    middle = int(len(examples)/2)
    train_data = data[:middle]
    test_data = data[middle:]
    test_file_name = file_name+".te"
    train_file_name = file_name+".tr"
    with open(test_file_name, 'w') as h:
        for ex in examples:
                h.write(ex.strip()+'\n')
    with open(train_file_name, 'w') as m:
        for ex in examples:
                m.write(ex.strip()+'\n')
    return

def take_top_n(labels_list,n):
    res = Counter(labels_list)
    items = list(res.items())
    sorted_list = sorted(items,key=lambda x: (x[1],x[0]),reverse=True)
    fir = sorted_list[:n-1]
    rest = sorted_list[n-1:]
    contry_code_list = [i[0] for i in fir]
    new_country_class = { name: k+1 for k,name in enumerate(contry_code_list) }
    return contry_code_list,new_country_class

def create_new_label(labels_list,n):# to reduce the number of classe to n
    contry_top_list,top_country_class=take_top_n(labels_list,n)
    new_label_list =[]
    for i in range(len(labels_list)):
        label = labels_list[i]
        if label in contry_top_list:
            new_label = top_country_class[label]
        else: 
            new_label = n
        new_label_list.append(new_label)
    return new_label_list

def add_labels(examples , labels_list):
    new_examples = []
    for i in range(len(labels_list)):
        ex = str(labels_list[i])
        ex +=' |%s' % examples[i]
        new_examples += [ex]
    return new_examples

In [68]:
with open("ukrain_loc.json","r") as f:
    data=json.load(f)

examples , labels_list = operation(data)


KeyboardInterrupt: 

In [58]:
new_label_list = create_new_label(labels_list,20)
print(new_label_list)
new_examples = add_labels(examples , new_label_list)
write_files("project/easter_sample",new_examples)

[12, 8, 20, 1, 1, 20, 9, 3, 9, 17, 9, 3, 1, 4, 20, 1, 5, 5, 5, 6, 20, 1, 4, 1, 9, 20, 1, 20, 20, 4, 3, 3, 5, 14, 3, 20, 20, 18, 5, 20, 1, 16, 3, 17, 3, 11, 1, 20, 3, 3, 1, 11, 11, 5, 2, 1, 20, 6, 15, 1, 20, 2, 12, 1, 20, 10, 1, 3, 1, 1, 20, 20, 2, 1, 1, 18, 20, 1, 13, 6, 7, 4, 2, 20, 1, 13, 8, 20, 8, 3, 6, 16, 5, 1, 9, 20, 20, 20, 18, 1, 1, 8, 5, 20, 20, 4, 11, 20, 7, 6, 1, 4, 7, 8, 1, 16, 6, 1, 1, 4, 9, 13, 20, 14, 1, 7, 19, 3, 5, 5, 1, 1, 1, 1, 1, 1, 15, 4, 20, 20, 16, 17, 1, 10, 5, 4, 2, 4, 3, 4, 20, 20, 8, 1, 10, 8, 20, 1, 12, 1, 7, 20, 1, 1, 1, 20, 1, 6, 20, 1, 1, 2, 6, 20, 2, 2, 19, 1, 2, 15, 1, 5, 1, 10, 1, 6, 1, 12, 4, 2, 4, 11, 2, 4, 7, 1, 1, 20, 1, 7, 6, 20, 1, 20, 20, 1, 1, 19, 6, 13, 7, 1, 1, 7, 20, 14, 20, 4, 3, 20, 3, 20, 12, 20, 2, 20, 2, 2, 14, 3, 8, 6, 9, 10, 10, 1, 20, 1, 2, 2, 15, 4, 6, 2, 2, 5, 20, 2, 7, 20, 1, 7, 7, 20, 2, 1, 13, 3, 3, 17, 20, 1, 12, 8, 20, 1, 1, 20, 9, 3, 9, 17, 9, 3, 1, 4, 20, 1, 5, 5, 5, 6, 20, 1, 4, 1, 9, 20, 1, 20, 20, 4, 3, 3, 5, 14, 3, 20, 2

In [64]:
!vw -k -c -b 27 --oaa 20 -d project/easter_sample.tr -f project/easter_sample.model --passes 20 --holdout_after 1000

final_regressor = project/easter_sample.model
Num weight bits = 27
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = project/easter_sample.tr.cache
Reading datafile = project/easter_sample.tr
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1       22
0.500000 1.000000            2            2.0        6        1       88
0.750000 1.000000            4            4.0       20        1       19
0.875000 1.000000            8            8.0       10        1       44
0.812500 0.750000           16           16.0        1        4       33
0.781250 0.750000           32           32.0        2        1       81
0.828125 0.875000           64           64.0        5        1       15
0.843750 0.859375          128          128.0        4        5       15
0.851562 0.859375 

In [50]:
contry_code_list,new_country_class=take_top_n(labels_list,20)
print(contry_code_list)
print(new_country_class)
print(new_country_class[239])

[239, 238, 14, 6, 175, 86, 234, 66, 182, 82, 197, 71, 83, 236, 157, 110, 40, 210, 183, 107, 104, 95, 59, 162, 132, 114, 109, 76, 50]
{239: 1, 238: 2, 14: 3, 6: 4, 175: 5, 86: 6, 234: 7, 66: 8, 182: 9, 82: 10, 197: 11, 71: 12, 83: 13, 236: 14, 157: 15, 110: 16, 40: 17, 210: 18, 183: 19, 107: 20, 104: 21, 95: 22, 59: 23, 162: 24, 132: 25, 114: 26, 109: 27, 76: 28, 50: 29}
1


In [45]:
from collections import Counter

res = Counter(labels_list)
items = list(res.items())
sorted_list = sorted(items,key=lambda x: (x[1],x[0]),reverse=True)
print(sorted_list[:29])
fir = sorted_list[:29]
rest = sorted_list[29:]
cont_code = [i[0] for i in fir]
print(cont_code)

[(239, 122), (238, 39), (14, 34), (6, 32), (175, 26), (86, 26), (234, 24), (66, 16), (182, 14), (82, 12), (197, 10), (71, 10), (83, 9), (236, 8), (157, 8), (110, 8), (40, 7), (210, 6), (183, 6), (107, 6), (104, 6), (95, 6), (59, 6), (162, 4), (132, 4), (114, 4), (109, 4), (76, 4), (50, 4)]
[239, 238, 14, 6, 175, 86, 234, 66, 182, 82, 197, 71, 83, 236, 157, 110, 40, 210, 183, 107, 104, 95, 59, 162, 132, 114, 109, 76, 50]


In [40]:
print(sorted_list[:][0])

(239, 122)


In [38]:
a=238
index_row = [cities.index(row) for row in sorted_list if a in row]
print(index_row)

ValueError: (238, 39) is not in list